In [1]:
import uniprot
import word2num

In [2]:
ds = uniprot.Dataset()
release, df = ds.load('sprot')

Workdir=/tmp/tmpz8_46g24


In [3]:
df.head()

,dataset,name,organism,proteinexistence,sequence
ID,,,,,
Q6GZX4,Swiss-Prot,001R_FRG3G,Frog virus 3 (isolate Goorha),4,MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQV...
Q6GZX3,Swiss-Prot,002L_FRG3G,Frog virus 3 (isolate Goorha),4,MSIIGATRLQNDKSDTYSAGPCYAGGCSAFTPRGTCGKDWDLGEQT...
Q197F8,Swiss-Prot,002R_IIV3,Invertebrate iridescent virus 3,4,MASNTVSAQGGSNRPVRDFSNIQDVAQFLLFDPIWNEQPGSIVPWK...
Q197F7,Swiss-Prot,003L_IIV3,Invertebrate iridescent virus 3,4,MYQAINPCPQSWYGSPQLEREIVCKMSGAPHYPNYYPVHPNALGGA...
Q6GZX2,Swiss-Prot,003R_FRG3G,Frog virus 3 (isolate Goorha),3,MARPLLGKTSSVRRRLESLSACSIFFFLRKFCQKMASLVFLNSPVY...


In [4]:
len(df)

561356

In [5]:
vocab, tok, ids = word2num.transform(df, lambda seq: [t for t in seq])

561356it [01:34, 5938.49it/s] 


In [10]:
tok[:1]

array([list([21, 17, 2, 15, 6, 2, 5, 10, 4, 1, 8, 5, 16, 10, 9, 9, 9, 9, 17, 5, 2, 1, 1, 1, 6, 1, 16, 16, 12, 13, 10, 9, 8, 1, 1, 10, 16, 8, 5, 20, 6, 12, 12, 9, 4, 14, 4, 5, 19, 12, 8, 2, 12, 4, 5, 20, 13, 13, 12, 12, 6, 5, 8, 3, 1, 7, 4, 3, 18, 15, 6, 3, 7, 8, 16, 8, 3, 5, 8, 2, 14, 2, 6, 5, 4, 10, 4, 13, 8, 17, 19, 19, 20, 4, 6, 8, 15, 8, 10, 2, 17, 9, 9, 16, 14, 3, 7, 14, 11, 19, 8, 7, 12, 3, 8, 4, 1, 6, 10, 1, 10, 2, 8, 7, 8, 2, 16, 13, 1, 11, 4, 5, 3, 4, 5, 3, 15, 4, 9, 16, 6, 9, 4, 11, 8, 14, 18, 4, 2, 2, 15, 1, 8, 5, 1, 9, 18, 6, 8, 14, 16, 5, 13, 4, 13, 1, 7, 18, 16, 7, 1, 11, 10, 8, 9, 4, 10, 7, 14, 18, 1, 5, 8, 10, 1, 4, 8, 10, 15, 8, 2, 1, 4, 5, 6, 2, 18, 9, 17, 9, 14, 3, 18, 17, 7, 13, 4, 8, 16, 7, 1, 16, 14, 1, 1, 8, 8, 18, 3, 18, 3, 12, 10, 3, 12, 10, 7, 1, 11, 4, 8, 11, 3, 6, 8, 3, 4, 1, 16, 10, 10, 6, 15, 9, 8, 7, 16, 11, 10, 1, 3, 20, 8, 15, 11, 12, 1])],
      dtype=object)